# Weight timeseries data analysis
### Introduction
Here we will analyze a (fake) dataset of the weight of twenty individuals over time. The data is split up into two files:

1. `weight_data.csv` with columns: `|| person | Day_1 | ... | Day_6 ||`
2. `weight_metadata.csv`: `|| person | sex | homestate | ... | feature_n ||`

The challenges involved with this assignment will include transforming, merging, and aggregating this data in different ways in order to visualize different aspects. This assignment is for someone who has had at least some exposure to [`pandas`](https://pandas.pydata.org/) and plotting with [`seaborn`](https://seaborn.pydata.org/index.html).

If you're interested in seeing how this data was generated, take a look [here](https://github.com/michaelsilverstein/Pandas-and-Plotting/blob/master/weight_timeseries/weight_timeseries_generate_data.ipynb).

**Note**: This exercise will likely require you to search for the syntax to accomplish each task. I have done my best to strike a balance between providing the vocabulary (and some syntax) to describe some of the concepts but I believe the best way to learn these in part is to try and find the documentation to the functions that meet your need. I will also try to include random opportunities to learn helpful syntax...

Jupyter lab tips:
- `ctrl` + `enter` will run cells
- `shift` + `tab` when your cursor is in a function will display the documentation
- output the value of a variable by having that variable be the last element of a cell
- **In general when manipulating data, print out your object at each step of manipulation**

### Created by
[Michael Silverstein](https://github.com/michaelsilverstein)

In [ ]:
"""Run this cell to load libraries!"""
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Brief Seaborn introduction
Before using `seaborn`, take a quick saunter around their [documenation](https://seaborn.pydata.org/). Here you will see the types of figures `seaborn` makes so easy. `Seaborn` is a "wrapper" of sorts - it's built on top of Python's plotting library [`matplotlib`](https://matplotlib.org/) and allows for easy implementation of plotting categorical data.

`Seaborn` accepts "longform" data structures as input in order to generalize the categorical assocation of each element of your data. [Take a look at my brief introduction to melting and pivotting here](https://github.com/michaelsilverstein/Pandas-and-Plotting/blob/master/lessons/melting_pivotting.ipynb) which are the methods of data transformation that allow you to shift data between longform and wideform types.

In order to illustrate `seaborn`'s syntax, we'll start off with a super short example that highlights the use of the `hue` parameter which allows you to indicate the set of categories to color your data by when plotting.

In [ ]:
"""Seaborn hue example"""
# Let's load the 'iris' dataset about different species of iris flowers
df = sns.load_dataset('iris')
df.head()

In [ ]:
# Now let's plot a scatter plot of two of these features coloring each dot by the species
plt.figure(figsize=(5, 5)) # Change the plot size (optional)
sns.scatterplot('sepal_length', 'petal_length', 'species', data=df)
plt.legend(bbox_to_anchor=(1, 1), ncol=2) # Move the legend and change number of columns (optional)
plt.show()

As you can see, when using `seaborn` we pass the **name** of the columns as parameters. If you [look at the `sns.scatterplot` documentation](https://seaborn.pydata.org/generated/seaborn.scatterplot.html) you'll see that this function, as almost all seaborn functions, accept `x`, `y`, and `hue`, as the first three arguments.

# Start
The "assignment" or whatever starts here. From here on out, we will operate with the synthetically generated weight over time data. We will load and then do some basic analysis and plotting which will require some transformations, merging, and aggregation.

I have created cells with comments for you to write your code. Typically, I will prepare my data in one cell, plot the head of that dataframe in the same cell, and then plot in the following cell.

In [ ]:
"""Load observed data"""
# Load wideform observed weight data with pandas
df = pd.read_csv('weight_data.csv')
df.head()

In [ ]:
# Load metadata
md = pd.read_csv('weight_metadata.csv')
md.head()

# Timeseries
Currently `df` is a wideform dataframe. (Again for a refresher on longform vs. wideform, check [here](https://github.com/michaelsilverstein/Pandas-and-Plotting/blob/master/lessons/melting_pivotting.ipynb)). Our current goal will be to plot timeseries data with the [`sns.lineplot`](https://seaborn.pydata.org/generated/seaborn.lineplot.html) function. In order to do this, we will want our dataframe to look like the following:

```python
melted.head()
```

| person | week | weight     |
|--------|------|------------|
| P0     | 0    | 167.743108 |
| P1     | 0    | 163.719962 |
| P2     | 0    | 175.474169 |
| P3     | 0    | 171.008108 |
| P4     | 0    | 167.238358 |


Notice that the `week` column contains the `int` **not** the `str` `Week_0`. If we were to extract the `0` from `Week_0` in standard python, we could do:
```python
int('Week_0'.split('_')[1])
```
In `pandas`, we can apply string operations like `split` to an entire column:
```python
# Split every element of the column `week` in the dataframe `melted` on '_', get the 1th element, and convert the result to `int`
melted['week'] = melted.week.str.split('_').str.get(1).astype(int)
```

Notice that we can make a new column (or overwrite an existing one) with `dataframe['column_name'] = some_array`.

Once our dataframe is melted, then we can plot with `sns.lineplot` to get the following:

![weight_timeseries](figures/weight_timeseries.png)

Although `seaborn` will automatically make a color palette for us, let's generate one now to ensure consistency color assignment for each plot. Be sure to check out how palettes work in the `seaborn` examples in the documentation.
```python
# Make a dictionary that will assign a color to each person
# Check out what md.person.unique() and md.person.nunique() produce
person_pal = dict(zip(md.person.unique(), sns.husl_palette(md.person.nunique())))
```

The circles (or `marker`s is bonus ;) )

In [ ]:
"""Melt dataframe"""
# Melt and convert 'week' to `int` and then print the head of the resulting dataframe

melted.head()

In [ ]:
"""Plot"""
# Plot weight over time for each individual by coloring by each individual

plt.show()

# Timeseries + metadata
We can already see some stratification in the data - there are clearly two larger groupings of types of people and then within each group it seems as if some people have gained weight and others have lost weight. 

Let's use the associated **metadata** to see if we can explore if there are any underlying trends. First we will have to [merge](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html) the metadata with our observed data. Merging allows you to combine two tables on their shared (or user-specified) columns. The head of your merged dataframe should look like:
```python
merged.head()
```

| person | week | weight     | sex  | state |
|--------|------|------------|------|-------|
| P0     | 0    | 167.743108 | male | MA    |
| P0     | 1    | 174.449221 | male | MA    |
| P0     | 2    | 173.804793 | male | MA    |
| P0     | 3    | 169.578463 | male | MA    |
| P0     | 4    | 172.608399 | male | MA    |


After merging we can use the resulting dataframe to add more information to our plot by using the `style` and `size` parameters:

![timeseries+md](figures/weight_timeseries+md.png)

### Questions
- `sns.lineplot` will default to using dashes for the column passed to the `style` column - how can we turn off the dashses and use markers?
- how can we change the thickness of the lines?

In [ ]:
"""Merge metadata"""
# Merge metadata to 'melted' and print the head


In [ ]:
"""Plot"""
# Plot the same as above but add 'sex' and 'weight' information as `size` and `style` respectively

plt.show()

# Comparing weight by sex
One feature (or column) that clearly stratifies the data is the `sex` and we could use a [`boxplot`](https://seaborn.pydata.org/generated/seaborn.boxplot.html) to visualize these differences more directly. If we are to properly comapre the differences in weight between sex we will have to do so with [independent](https://en.wikipedia.org/wiki/Independence_(probability_theory)) datapoints - which in our case means that we won't want to have multiple datapoints per person since these data are indeed dependent on each other. 

There are multiple approaches we could take to generate an independent dataset from our current one. We could choose just one timepoint (sample from our data) or aggregate our data in some way, like by summing or averaging the data from each person. For this example let's take the mean weight from each person to investigate the effect sex has on weight.

We can take the weight in two ways:
```python
"""1) Use the original dataframe"""
# Set 'person' as the index and then take the mean over the rows (axis=1) (Note: the default is axis=0, the columns)
#  Reset index and name it 'weight' and then we can `merge` out metadata to the resulting dataframe
means = df.set_index('person').mean(1).reset_index(name='weight')
"""2) Use melted dataframe with groupby"""
# Group data by 'person', take the mean of each group's weight, and then reset the index. We can then add our metadata back
# We want to reset the index so that we can pass it as a column to our plotting functions
means = merged.groupby('person').weight.mean().reset_index()
# An alternative that illustrates grouping by multiple columns and avoids needing to re-merge our metadata:
means = merged.groupby(['person', 'sex', 'state']).week.mean().reset_index()
```
Both of the above will produce the same result. I encourage you to look at the [`pd.groupby` documentation](https://pandas.pydata.org/pandas-docs/version/0.22/groupby.html) and to try and approach this challenge using it. This is just a measley introduction to `pd.groupby`, but it is an endlessly valuable tool and a complicated one to wrap your mind around at first, so I hope this can serve as a simple introduction.

After getting the mean weights, we can plot resulting data in a boxplot and then also overlay the underlying data using [`sns.swarmplot`](https://seaborn.pydata.org/generated/seaborn.swarmplot.html). When you do this, you should get a plot that looks like:

![mean_boxplot](figures/mean_weight_sex.png)

### Questions
- How can we change the colors from their default for the boxplot? $\implies$ Check out the `palette` parameter
- For the swarmplot?

In [ ]:
"""Take the mean weight of each person over the 6 weeks"""
# Take mean weight and merge the metadata for each person and then print the head of the resulting dataframe


In [ ]:
"""Plot box plot"""
# Let's do a boxplot

# Then let's do a swarmplot

# Now when we plot they will be overlayed

plt.show()

## By state and then sex
This clearly is a difference between the weight and the males and females here! Let's break this data down further to see if there are further relations. Since these people are from different states, we may want to control for which state they are from in our analysis, so let's break the data down by state and then by sex:

![boxplot_state](figures/mean_weight_state.png)

### Questions:
- How do we align the `swarmplot` points with the boxes when using a `hue`? $\implies$ Check out the `dodge` parameter

In [ ]:
"""Plot by state and then sex"""
# Boxplot

# Swarmplot

plt.show()

# Timeseries $\Delta$ Baseline
If we were interested in which person gained or lost "the most weight", what we really may be asking is who changed the most from their baseline? To do this, will have to calculated the change in each person's baseline, which in this case is their weight at week 0. We can do this in a myraid of ways, but my preferred method is the following because it allows you to see all of the data involved in your cacluation:

| person | week | weight     | sex  | state | baseline   | delta    |
|:------:|-----|------------|------|-------|------------|----------|
|   P0   |    0 | 167.743108 | male | MA    | 167.743108 | 0.000000 |
|   P0   |    1 | 174.449221 | male | MA    | 167.743108 | 6.706113 |
|   P0   |    2 | 173.804793 | male | MA    | 167.743108 | 6.061685 |
| P0     | 3    | 169.578463 | male | MA    | 167.743108 | 1.835354 |
| P0     | 4    | 172.608399 | male | MA    | 167.743108 | 4.865290 |

After merging the baseline of each patient to our dataframe, we can compute the delta column above with:
```python
merged['delta'] = merged.weight - merged.baseline
```
In general, `pandas` allows us to perform vector operations on [`Series`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html) (the information in each column), so we can easily operate on one or multiple columns as vectors.

Now, we can plot the change from baseline, much like how we plotted the absolute weight before:
![baseline_timeseries](figures/baseline_timeseries.png)
### Questions
- How can we get the black line at zero? $\implies$ check out [`plt.axhline`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.axhline.html)
- How could we plot the percent change from baseline?

## Challenge
How can we color by sex?

![baseline_colored](figures/baseline_timeseries_colored.png)

I've put some hints in the cell below

In [ ]:
"""Compute difference from baseline weight"""
# First, let's get the baseline weight for each person

# Next, let's merge that with out dataframe

# Now let's calculate the difference between the weight at each point and the baseline


In [ ]:
"""Plot change from baseline timeseries"""
# Using lineplot, let's plot the change from baseline timeseries

plt.show()

In [ ]:
"""Challenge: Plot change from baseline timeseries and color by sex"""
# That's a pretty busy plot. Although we are using the line thickness to delineate sex, let's make differences
# more clear by using different colors for each sex
# We assign colors using the `palette` argument which chooses a color for each element of the `hue`

plt.show()

# Timeseries facetted by state
That is some busy shit going on up there... Like before with the boxplots, let's control our analysis for which state each person is from by breaking up these timeseries by state by using the [extremely powerful function sns.FacetGrid](https://seaborn.pydata.org/generated/seaborn.FacetGrid.html). This function allows us to create a grid of plots by choosing a column to "facet" on - just like how we can use `seaborn` to color by information in one column, `FacetGrid` will allow us to choose a column to make individual plots in a grid with. This is a miserable description - it's close to dinner time and I'm hungry - please look at the documentation.

Facetting change in baseline by state we get

![facet_state](figures/facet_timeseries.png)

### Questions
1. How can we get only two plots per row?
2. How can we get each plot to be wider?
3. How can we get the black zero line on each plot?

For scatter and lineplots, you can also use the [`sns.relplot`](https://seaborn.pydata.org/generated/seaborn.relplot.html) to facet, but I wanted to show you `FacetGrid` since it is more generalizable.

In [ ]:
"""Facet change in baseline timeseries by state"""
# Pass our dataframe to FacetGrid and facet by 'state'

plt.show()